In [1]:
USER_KEY = "cc3948f0cd364a9f684a1fe78b9229ba"      #  Input your User key
DATA_KEY = "a71eaf04-802f-40be-93c2-5bee2548f4db"  #  Fixed Data Api key

print("USER_KEY:", USER_KEY if USER_KEY else "Not found")
print("DATA_KEY:", DATA_KEY if DATA_KEY else "Not found")

USER_KEY: cc3948f0cd364a9f684a1fe78b9229ba
DATA_KEY: a71eaf04-802f-40be-93c2-5bee2548f4db


In [82]:
import requests

root_url = f'https://aifapbt.fin.cloud.ainode.ai/'

headers = {"API-KEY": USER_KEY}
requests.get(root_url, headers=headers).json()

{'message': 'API Server is Running.',
 'status': 200,
 'User status': 'active',
 'Expires At': '2025-10-01T00:00:00',
 'Level': 1}

In [83]:
import os
strategy_name = "simple_reversion"

# Upload or Update strategy file
endpoint = 'upload/strategy'
url = root_url + endpoint

base_path = "/Users/seongmin/Desktop/neomatrix_trading/futures"
file_path = os.path.join(base_path, strategy_name, strategy_name + ".py")

params = {'tradeType':'futures', 'overwrite': True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'simple_reversion' Strategy Updated Successfully!"}


In [84]:
# Check strategy file upload
endpoint = 'upload/check/strategy'
url = root_url + endpoint

params = {"tradeType": "futures", "strategy_name": strategy_name}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# strategy.py
import pandas as pd
import numpy as np

def strategy(df, config_dict):
    """
    Generates a mean-reversion signal based on historical cumulative returns.

    The signal is the negative of the cumulative return over a lookback period,
    ignoring the most recent 'lag' periods. This strategy sells recent
    winners and buys recent losers.
    """
    # --- Input validation ---
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame.")
    if df.empty:
        raise ValueError("Input DataFrame is empty.")
    if not isinstance(config_dict, dict):
        raise TypeError("config_dict must be a dictionary.")

    # --- Load strategy-specific config ---
    strategy_specific_config = config_dict.get("strategy_config", {})
    # Lookback period in hours for the return calculation
    lookback_hours = strategy_specific_config.get("lookback_hours", 10)
    # Number of recent hours to ignore in the calculation
    lag_hours = strateg

In [85]:
# Upload or Update strategy-config file
endpoint = 'upload/strategy-config'
url = root_url + endpoint

strategy_config_name = strategy_name + "_config"
base_path = "/Users/seongmin/Desktop/neomatrix_trading/futures/"
file_path = os.path.join(base_path, strategy_name, strategy_config_name + ".py")

params = {"tradeType": "futures", "overwrite": True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'simple_reversion' Strategy-Config Updated Successfully!"}


In [86]:
# Check strategy-config file upload
endpoint = 'upload/check/strategy-config'
url = root_url + endpoint

params = {"tradeType": "futures", "strategy_name": strategy_name}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# strategy_config.py

strategy_config = {
    "lookback_hours": 10,  # Lookback period for cumulative return calculation
    "lag_hours": 1         # How many recent hours to ignore from the lookback
}


In [87]:
# Upload or Update system-config file
endpoint = "upload/system-config"
url = root_url + endpoint

system_config_name = "futures_config"
base_path = "/Users/seongmin/Desktop/neomatrix_trading/futures"
file_path = os.path.join(base_path, system_config_name + ".py")

params = {"tradeType": "futures", "overwrite": True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'Futures' System-Config Updated Successfully!"}


In [88]:
# Check system-config file upload
endpoint = "upload/check/system-config"
url = root_url + endpoint

params = {"tradeType": "futures"}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# ==========================
# Required System Settings
# ==========================
system_config = {
    "data_apikey": "a71eaf04-802f-40be-93c2-5bee2548f4db",
    "tz_str": "Asia/Seoul", 
    "timeframe": "1min",
    "orderType": "market",
    "base_symbol": "BTCUSDT",
    "productType": "usdt-futures",
    "posMode": "hedge_mode",
    "holdSide": "long",                   
    "marginMode": "crossed",
    "marginCoin": "usdt",
    "symbols": ['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'TRXUSDT', 'DOTUSDT', 'DOGEUSDT', 'XRPUSDT', 'BCHUSDT', 'ADAUSDT'],
    "leverage": 5,
    "trading_hours": 6,
    "total_allocation": 0.4, 
}


# ==========================
# Rebalancing Trade Parameters
# ==========================
rebalancing_config = {
    "rebalancing_interval_hours": 1, # Rebalancing cycle (hours)
}



In [89]:
# run-system
endpoint = 'command/run-system'
url = root_url + endpoint

data = {
    "tradeType": "futures",
    "strategy_name": strategy_name,
    "trade_mode": "rebalancing",
    "trade_env": "live"
}

response = requests.post(url, headers=headers, json=data)

try:
    session_id = response.json()['session_id']
    print('session_id :',response.json()['session_id'])
    print('dashboard :',response.json()['dashboard_url'])
except:
    print('Error :',response.json()['message']['message'])

    session_id = response.json()['message']['session_id']
    print('session_id id :',session_id)

    print('dashboard :',response.json()['message']['dashboard_url'])

session_id : 7e07128e
dashboard : https://aifapbt.fin.cloud.ainode.ai/dashboard/live_futures_trading_dashbaord.html


In [80]:
# Terminate process
# You must execute terminate to force liquidation of the current process.
endpoint = 'command/terminate'
url = root_url + endpoint

params = {"session_id": f"{session_id}", "tradeType": 'futures'}

response = requests.get(url, headers=headers, params=params)
print(response.json())

{'message': 'Session d407b770 is terminated.'}
